The dataset is in kaggle, I downloaded it and splitted the dataset for training, in this file there are some operations i had to do to fix some inconsistencies, example of how i separated data:


# Explanation: Splitting by patient diagnosis means:
# - Aggregating diagnosis per patient (max(diagnosis) = 1 if any slice is positive).
# - Using stratified split to ensure similar proportions of positive/negative PATIENTS
#   in train/val/test (avoids imbalance and data leakage across patients).
# - This is done with stratify=patient_diagnosis['diagnosis'] in train_test_split.

# Stratified split by patient diagnosis (70% train, 15% val, 15% test)
train_patients, temp_patients = train_test_split(patient_diagnosis, test_size=0.3, stratify=patient_diagnosis['diagnosis'], random_state=42)
val_patients, test_patients = train_test_split(temp_patients, test_size=0.5, stratify=temp_patients['diagnosis'], random_state=42)

# Now, filter the full DF based on patient splits
train_df = df[df['patient'].isin(train_patients['patient'])] # in train_df inserisco tutti i
val_df = df[df['patient'].isin(val_patients['patient'])]
test_df = df[df['patient'].isin(test_patients['patient'])]

# Reset indices
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [5]:


import pandas as pd
import os

data_dir = '/home/stud/fmarchetto/SegmentationTests/Data/'

csv_files = ['balanced_train_df.csv', 'test_df.csv', 'val_df.csv']

old_prefix = '/home/stud/fmarchetto/SegmentationTests/data/'  # Lowercase 'data'
new_prefix = '/home/stud/fmarchetto/SegmentationTests/Data/'  # Capital 'Data'

for csv_file in csv_files:
    # Full paths
    full_csv_path = os.path.join(data_dir, csv_file)
    backup_path = os.path.join(data_dir, csv_file + '.bak')
    
    if not os.path.exists(full_csv_path):
        print(f"File {full_csv_path} not found. Skipping.")
        continue
    
    if not os.path.exists(backup_path):
        os.rename(full_csv_path, backup_path)
        print(f"Backed up {csv_file} to {backup_path}")
    else:
        print(f"Backup {backup_path} already exists—skipping backup for {csv_file}")
    
    # Load the CSV from the backup
    df = pd.read_csv(backup_path)
    
    # Identify columns ending with 'path'
    path_columns = [col for col in df.columns if col.endswith('path')]
    
    if not path_columns:
        print(f"No columns ending with 'path' found in {csv_file}. Skipping.")
        continue
    
    # Replace the string in each path column
    for col in path_columns:
        df[col] = df[col].str.replace(old_prefix, new_prefix, regex=False)
    
    # Save the updated DataFrame back to the original filename
    df.to_csv(full_csv_path, index=False)
    print(f"Updated {csv_file} successfully.")
    
    # Verify: Print a sample updated path (first row)
    if not df.empty:
        sample_path = df[path_columns[0]].iloc[0]  # e.g., first image_path
        if os.path.exists(sample_path):
            print(f"Verification: Sample path '{sample_path}' exists.")
        else:
            print(f"Warning: Sample path '{sample_path}' does NOT exist—check data directory.")

print("All files processed.")

Backed up balanced_train_df.csv to /home/stud/fmarchetto/SegmentationTests/Data/balanced_train_df.csv.bak
Updated balanced_train_df.csv successfully.
Verification: Sample path '/home/stud/fmarchetto/SegmentationTests/Data/kaggle_3m/TCGA_HT_7475_19970918/TCGA_HT_7475_19970918_12.tif' exists.
Backed up test_df.csv to /home/stud/fmarchetto/SegmentationTests/Data/test_df.csv.bak
Updated test_df.csv successfully.
Verification: Sample path '/home/stud/fmarchetto/SegmentationTests/Data/kaggle_3m/TCGA_DU_5854_19951104/TCGA_DU_5854_19951104_1.tif' exists.
Backed up val_df.csv to /home/stud/fmarchetto/SegmentationTests/Data/val_df.csv.bak
Updated val_df.csv successfully.
All files processed.


In [13]:
!find /home/stud/fmarchetto/SegmentationTests/Data/kaggle_3m -type f | wc -l



7860
